In [24]:
import matplotlib.pyplot as plt
import numpy as np
def gen_error_poly(theory_volts,adc_volts,verbose=False):
    # numpyfy
    setvolts = np.array(theory_volts)
    adcvolts = np.array(adc_volts)

    absolute_error = setvolts - adcvolts

    errorpoly = np.polyfit(adcvolts, absolute_error, 3)
    errorfunc = np.poly1d(errorpoly)

    corrected = adcvolts + errorfunc(adcvolts)
    corr_error = setvolts - corrected

    if (verbose):
        plt.rcParams["figure.figsize"] = (10,16)
        plt.figure()
        plt.subplots_adjust(wspace=0.5, hspace=0.5)
        plt.subplot(411)
        plt.plot(adcvolts,setvolts)
        plt.xlabel('Measurement [Volts]')
        plt.ylabel('Setting [Volts]')
        plt.title('Measurement vs. Setting')
        plt.subplot(412)

        plt.plot(adcvolts, absolute_error)
        plt.plot(adcvolts, errorfunc(adcvolts))
        plt.xlabel('Measurement [Volts]')
        plt.ylabel('Absolute Error')
        plt.title('Measurement Error')
        plt.subplot(413)

        plt.xlabel('Measurement [Volts]')
        plt.ylabel('Corrected [Volts]')
        plt.title('After Correction')
        plt.plot(adcvolts, corrected)
        plt.subplot(414)

        plt.xlabel('Measurement [Volts]')
        plt.ylabel('Absolute Error')
        plt.title('Error After Correction')
        plt.plot(adcvolts, corr_error)
    return errorpoly
    

In [46]:
# J18, J19, and J24 must be closed
# Nothing should be connected to the Target connector or J25
import pyvisa
import time
from pynq import Overlay
from fobos.software.pynq_conf import IP, PORT, OVERLAY_FILE
from fobos.software.foboslib.power import PowerManager
import pandas as pd 

def getinputCal(src, mode):
    while(1):
        rec = input("Connect {} OUT to DC load then enter y, or enter n to skip {} calibration:".format(src, mode))

        if (rec.strip("\n").lower() == "y"):
            return True
        if (rec.strip("\n").lower() == "n"):
            return False

overlay = Overlay(OVERLAY_FILE)
pm = PowerManager(overlay, False)


##### CALIBRATION SETTINGS #####
MAX_MILLIAMPS = 350
NUM_STEPS = 10

MAX_VOLTS = 3.5
MIN_VOLTS = 1
STEP_VOLTS = .25
VOLT_CURR_MILLIAMPS = 50

SAMPLES_PER_STEP = 50
WAIT_SECONDS = 5

DC_LOAD_SCPI_ADDR = 'TCPIP0::192.168.10.21::INSTR'
data = pd.DataFrame(columns=["source","gain","type","poly"])

##### Connect and init connection to DC Load #####
rm = pyvisa.ResourceManager()

# Connect

try:
    # Init
    my_instrument = rm.open_resource(DC_LOAD_SCPI_ADDR)
    my_instrument.read_termination = '\n'
    my_instrument.write_termination = '\n'
    print(my_instrument.query('*IDN?'))

    # Set to Constant Current (CC) mode
    my_instrument.write(':SOUR:FUNC CURRent')
except:
    my_instrument.close()
    
    
GAINS = [25]
# GAINS = [25, 50, 100, 200]
# NAMES = ["VAR", "3V3","5V"]
NAMES = ["VAR", "5V", "3V3"]


#### Run VAR voltage calibration #####
if (getinputCal("VAR", "voltage")):
    pm.OutVarOn()
    pm.OutVarSet(1)
    my_instrument.write(':SOURce:CURRent:LEVel:IMMediate {}'.format(0))
    print("Running volage calibration for VAR power supply")
    actual_volts = []
    pm.OutVarSet(MIN_VOLTS)
    theory_volts = [x/100 for x in list(range(int(MIN_VOLTS*100), int(MAX_VOLTS*100), int(STEP_VOLTS*100)))]
    for volt in theory_volts:
        pm.OutVarSet(volt)
        time.sleep(WAIT_SECONDS)

        meas = 0
        for i in range(SAMPLES_PER_STEP):
            meas += pm.MeasVoltVar()
        meas = meas/SAMPLES_PER_STEP
        actual_volts.append(meas)

        print("Target Voltage", volt, "Measured Voltage:", actual_volts[-1])

    err_poly = gen_error_poly(theory_volts, actual_volts, False)
    print(err_poly)

    for GAIN in GAINS:
        line = ["VAR", GAIN, "VOLT", list(err_poly)]
        data.loc[len(data)] = line

##### Connect to FOBOS control board #####
pm.OutVarOn()
pm.OutVarSet(1)
    
    
#### Run current calibration #####
for NAME in NAMES:
    if NAME == "VAR":
        cont = getinputCal("VAR", "current")
    elif NAME == "5V":
        cont = getinputCal("5V", "current")
    elif NAME == "3V3":
        cont = getinputCal("3V3", "current")
        
    shield_volts = []
    load_volts   = []
    load_currs   = []
    if (cont):
        for GAIN in GAINS:
            pm.GainVarSet(GAIN)
            print("Running current calibration for {} power supply for gain={}".format(NAME, GAIN))


            # ##### Gather samples #####
            print("Code, ADC Current, Actual Current")
            theory_amps = [x/1000 for x in list(range(0,MAX_MILLIAMPS,MAX_MILLIAMPS//NUM_STEPS))]
            shield_amps = []

            print("Currents steps:", theory_amps)

            for amp in theory_amps:
                actual_curr = amp

                my_instrument.write(':SOURce:CURRent:LEVel:IMMediate {}'.format(actual_curr))
                time.sleep(WAIT_SECONDS)
                
                actual_curr = float(my_instrument.query(':MEASure:CURRent:DC?'))
                actual_volt = float(my_instrument.query(':MEASure:VOLTage:DC?'))
                
                load_currs.append(amp)
                load_volts.append(actual_volt)
                
                if NAME == "VAR":
                    meas = 0
                    for i in range(SAMPLES_PER_STEP):
                        meas += pm.MeasCurrVar()
                    meas = meas/SAMPLES_PER_STEP
                    shield_amps.append(meas)
                    print(i, ",", meas, "," ,  actual_curr)    
                elif NAME == "5V":
                    meas = 0
                    for i in range(SAMPLES_PER_STEP):
                        meas += pm.MeasCurr5v()
                    meas = meas/SAMPLES_PER_STEP
                    shield_amps.append(meas)
                    print(i, ",", meas, "," ,  actual_curr)    
                    
                    meas = 0
                    for i in range(SAMPLES_PER_STEP):
                        meas += pm.MeasVolt5v()
                    meas = meas/SAMPLES_PER_STEP
                    shield_volts.append(meas)
                elif NAME == "3V3":
                    meas = 0
                    for i in range(SAMPLES_PER_STEP):
                        meas += pm.MeasCurr3v3()
                    meas = meas/SAMPLES_PER_STEP
                    shield_amps.append(meas)
                    print(i, ",", meas, "," ,  actual_curr)    
                    
                    meas = 0
                    for i in range(SAMPLES_PER_STEP):
                        meas += pm.MeasVolt3v3()
                    meas = meas/SAMPLES_PER_STEP
                    shield_volts.append(meas)

            my_instrument.write(':SOURce:CURRent:LEVel:IMMediate {}'.format(0))
            print(shield_amps, load_currs)
            cerr_poly = gen_error_poly(shield_amps,  load_currs, False)
            line = [NAME, GAIN, "CURR", list(cerr_poly)]
            data.loc[len(data)] = line
            
            if NAME != "VAR":
                print(shield_volts, load_volts)
                verr_poly = gen_error_poly(shield_volts, load_volts, False)
                line = [NAME, GAIN, "VOLT", list(verr_poly)]
                data.loc[len(data)] = line


    
##### Close Connection #####

my_instrument.close()
pm.OutVarOff()

data.to_csv("fobos/software/power_conf.csv", index=False)

    
print("done") 

RIGOL TECHNOLOGIES,DL3021A,DL3A234601185,00.01.05.00.01
Connect VAR OUT to DC load then enter y, or enter n to skip voltage calibration:y
Running volage calibration for VAR power supply
Target Voltage 1.0 Measured Voltage: 0.8763302052338442
Target Voltage 1.25 Measured Voltage: 1.1063889524681474
Target Voltage 1.5 Measured Voltage: 1.3382711528191045
Target Voltage 1.75 Measured Voltage: 1.5822354467078663
Target Voltage 2.0 Measured Voltage: 1.828667124437323
Target Voltage 2.25 Measured Voltage: 2.063291371023119
Target Voltage 2.5 Measured Voltage: 2.311750972762647
Target Voltage 2.75 Measured Voltage: 2.5612954909590298
Target Voltage 3.0 Measured Voltage: 2.805302510109101
Target Voltage 3.25 Measured Voltage: 3.05325246051728
[ 0.0089202  -0.06921869  0.19147218  0.00410696]
Connect VAR OUT to DC load then enter y, or enter n to skip current calibration:y
Running current calibration for VAR power supply for gain=25
Code, ADC Current, Actual Current
[0.0, 0.025, 0.05, 0.075, 0.

In [47]:
def GetCalibration(src, mtype, gain):
    calibration_df = pd.read_csv("fobos/software/power_conf.csv") 
    cond = (calibration_df['source'] == src) & (calibration_df['gain'] == gain) & (calibration_df['type'] == mtype)
    if len(calibration_df.loc[cond].values.tolist()) < 1:
        return [0,0,0,0]
    coeffs = calibration_df.loc[cond].values.tolist()[0][3]
    return coeffs

print(GetCalibration("VAR", "VOLT", 25))
print(GetCalibration("VAR", "VOLT", 50))
print(GetCalibration("VAR", "VOLT", 100))
print(GetCalibration("VAR", "VOLT", 200))

print(GetCalibration("VAR", "CURR", 25))
print(GetCalibration("VAR", "CURR", 50))
print(GetCalibration("VAR", "CURR", 100))
print(GetCalibration("VAR", "CURR", 200))

print(GetCalibration("5V", "CURR", 25))
print(GetCalibration("5V", "CURR", 50))
print(GetCalibration("5V", "CURR", 100))
print(GetCalibration("5V", "CURR", 200))

print(GetCalibration("5V", "VOLT", 25))
print(GetCalibration("5V", "VOLT", 50))
print(GetCalibration("5V", "VOLT", 100))
print(GetCalibration("5V", "VOLT", 200))

print(GetCalibration("3V3", "CURR", 25))
print(GetCalibration("3V3", "CURR", 50))
print(GetCalibration("3V3", "CURR", 100))
print(GetCalibration("3V3", "CURR", 200))

print(GetCalibration("3V3", "VOLT", 25))
print(GetCalibration("3V3", "VOLT", 50))
print(GetCalibration("3V3", "VOLT", 100))
print(GetCalibration("3V3", "VOLT", 200))

[0.008920203126369824, -0.06921868786705627, 0.1914721773806424, 0.004106964054102036]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[-0.39186098262230445, -0.03848318922284529, 0.2063766377136037, 0.00045296623043963006]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]


In [49]:
import pyvisa
import time
from pynq import Overlay
from fobos.software.pynq_conf import IP, PORT, OVERLAY_FILE
from fobos.software.foboslib.power import PowerManager

## Connect and init connection to DC Load
rm = pyvisa.ResourceManager()
rm.list_resources()
my_instrument = rm.open_resource('TCPIP0::192.168.10.21::INSTR')
my_instrument.read_termination = '\n'
my_instrument.write_termination = '\n'
print(my_instrument.query('*IDN?'))

# Set to Constant Current (CC) mode
my_instrument.write(':SOUR:FUNC CURRent')
print(my_instrument.query(':SOUR:FUNC?'))


overlay = Overlay(OVERLAY_FILE)
pm = PowerManager(overlay, False)

pm.Reset()
pm.OutVarOn()
pm.OutVarSet(1)
pm.GainVarSet(25)

try:
    #### START DC LOAD SCRIPT
    currents = [.05,.15,.2,.25,.2,.15,.05]
    pm.TrigSwEnOn()
    for curr in currents:
        my_instrument.write(':SOURce:CURRent:LEVel:IMMediate {}'.format(curr))
        time.sleep(2)
        print("=======")
        print("Load Current:", my_instrument.query(':MEASure:CURRent:DC?'), "Measured Current:", pm.MeasCurrVar())
        print("Load Voltage:", my_instrument.query(':MEASure:VOLTage:DC?'), "Measured Voltage:", pm.MeasVoltVar())
    pm.TrigSwEnOff()
    #### END DC LOAD SCRIPT
            
    pm.OutVarOff()
    avg_curr = pm.MeasAvgCurrVar()
    avg_volt = pm.MeasAvgVoltVar()
    
    print("Avg curr:", avg_curr, "Avg volt:", avg_volt)
    print("Max curr:", pm.MeasMaxCurrVar(), "Max volt:", pm.MeasMaxVoltVar())
    print("expected:", sum(currents)/len(currents))  
finally:
    my_instrument.close()
    

RIGOL TECHNOLOGIES,DL3021A,DL3A234601185,00.01.05.00.01
CC
Load Current: 0.048635 Measured Current: 0.03875791561760891
Load Voltage: 0.991902 Measured Voltage: 0.871976806286717
Load Current: 0.148304 Measured Current: 0.11932555123216601
Load Voltage: 0.977192 Measured Voltage: 0.8643472953383688
Load Current: 0.199417 Measured Current: 0.1681239032578012
Load Voltage: 0.969659 Measured Voltage: 0.8583962767986572
Load Current: 0.248564 Measured Current: 0.208194094758526
Load Voltage: 0.962393 Measured Voltage: 0.8543526359960326
Load Current: 0.199321 Measured Current: 0.16433966582742046
Load Voltage: 0.969659 Measured Voltage: 0.8581673914702067
Load Current: 0.148280 Measured Current: 0.12433051041428245
Load Voltage: 0.977192 Measured Voltage: 0.8619058518348974
Load Current: 0.048611 Measured Current: 0.041992828259708556
Load Voltage: 0.991902 Measured Voltage: 0.8704509040970474
Sample Counter has overflown. Value will not be correct.
Sample Counter has overflown. Value will

TODO:
- [X] No difference for gains in voltage calibration
- [X] Use $FOBOS_HOME/software for path to calibration in power.py
- [X] Move calibration CSV to same folder as pynq_conf 
- [ ] Create requirement-pynq.txt for [pandas, pyvisa] AND call from install script
- [ ] Check that lack of CSV does not cause power.py to error out
- [ ] 5v/3v3 calibration over shunt once wire is ready (sense cables to get better V measurement)
- [ ] Move script to command line



RIGOL TECHNOLOGIES,DL3021A,DL3A234601185,00.01.05.00.01
Connect VAR OUT to DC load then enter y, or enter n to skip voltage calibration:y
Running volage calibration for VAR power supply
Target Voltage 1.0 Measured Voltage: 0.8757747768368048
Target Voltage 1.25 Measured Voltage: 1.1062790875104906
Target Voltage 1.5 Measured Voltage: 1.3382299534599837
Target Voltage 1.75 Measured Voltage: 1.581866178377966
Target Voltage 2.0 Measured Voltage: 1.8283192187380777
Target Voltage 2.25 Measured Voltage: 2.06328374151217
Target Voltage 2.5 Measured Voltage: 2.311476310368506
Target Voltage 2.75 Measured Voltage: 2.5607721065079723
Target Voltage 3.0 Measured Voltage: 2.805282673380635
Target Voltage 3.25 Measured Voltage: 3.0529213397421215
[ 0.00865651 -0.06754562  0.1881478   0.00639055]
Connect VAR OUT to DC load then enter y, or enter n to skip current calibration:y
Running current calibration for VAR power supply for gain=25
Code, ADC Current, Actual Current
Currents steps: [0.0, 0.035

In [21]:
shield_volts

[4.807466239414051, 4.782136263065535]

In [22]:
load_volts

['4.997792', '4.964309']

In [15]:
pm = PowerManager(overlay, False)


print(pm.MeasVolt5v())

pm.OutVarOff()

4.801861600671397
